необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
import re

In [4]:
data = data.replace('\?', np.nan, regex=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
# обнуляем ячейки с отсутствием данных
data = data[ pd.isnull( data['workclass'] ) == 0 ]
data = data[ pd.isnull( data['occupation'] ) == 0 ]
data = data[ pd.isnull( data['native-country'] ) == 0 ]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [9]:
# некоторые столбцы  являются категориальными переменными
# переведем их в значения 0 и 1, добавив столбцы с соответствующими названиями
data = pd.get_dummies(data, columns = [ 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country'] )

In [10]:
data.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,Male,0,0,40,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,Male,0,0,50,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,Male,0,0,40,>50K,0,1,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,Male,7688,0,40,>50K,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,Male,0,0,30,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
#осталось поработать со столбцами gender и income
# переведем значения столбцов в числа

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
le.fit( data['gender'] )
data['gender'] = le.transform( data['gender'] )

In [13]:
le.fit( data['income'] )
data['income'] = le.transform( data['income'] )

In [14]:
data.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,1,0,0,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,1,0,0,40,1,0,1,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,1,7688,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,1,0,0,30,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Логистическая регрессия

In [15]:
# записываем в переменную y преобразованный столбец income
y = pd.Series(data['income'])
y.head()

0    0
1    0
2    1
3    1
5    0
Name: income, dtype: int32

In [16]:
#столбец income является целевой переменной, удаляем его из data
del data['income']
data.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,1,0,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,1,0,0,50,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,1,0,0,40,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,1,7688,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,1,0,0,30,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


model = make_pipeline(StandardScaler(),LogisticRegression())

In [19]:
model = LogisticRegression()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)

In [21]:
# обучаем модель
model.fit( X_train, y_train )
predictions = model.predict( X_test )

In [24]:
model.score(X_train, y_train)

0.7896066972042332

In [25]:
model.score(X_test, y_test)

0.7954595710179111

In [22]:
from sklearn.metrics import accuracy_score

In [26]:
accuracy_score(y_test, predictions)

0.7954595710179111

модель логистической регрессии корректно предсказывает уровень дохода с достоверностью 79,5% 

SVM

In [27]:
from sklearn.svm import SVC


clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [28]:
clf.score(X_train, y_train)

0.8583478123519191

In [29]:
clf.score(X_test, y_test)

0.850961892828186

Метод SVM намного лучше работает, уровень достоверности предсказания составляет 85,1%